# Load and Merge All Data

In [1]:
import pandas as pd
from pathlib import Path

data_dir = Path("data")
all_csv_files = list(data_dir.rglob("*.csv"))

dfs = []
for f in all_csv_files:
    df = pd.read_csv(f)
    df["station_folder"] = f.parent.name
    df["file_name"] = f.name
    dfs.append(df)

data = pd.concat(dfs, ignore_index=True)

In [2]:
print(data.columns.tolist())
print(data.head())

['Date/Time', 'Electricity:Facility [kW](Hourly)', 'Fans:Electricity [kW](Hourly)', 'Cooling:Electricity [kW](Hourly)', 'Heating:Electricity [kW](Hourly)', 'InteriorLights:Electricity [kW](Hourly)', 'InteriorEquipment:Electricity [kW](Hourly)', 'Gas:Facility [kW](Hourly)', 'Heating:Gas [kW](Hourly)', 'InteriorEquipment:Gas [kW](Hourly)', 'Water Heater:WaterSystems:Gas [kW](Hourly)', 'station_folder', 'file_name', 'Electricity:Facility [kW](Monthly)', 'Gas:Facility [kW](Monthly)']
          Date/Time  Electricity:Facility [kW](Hourly)  \
0   01/01  01:00:00                          22.035977   
1   01/01  02:00:00                          14.649757   
2   01/01  03:00:00                          14.669567   
3   01/01  04:00:00                          14.677808   
4   01/01  05:00:00                          14.824794   

   Fans:Electricity [kW](Hourly)  Cooling:Electricity [kW](Hourly)  \
0                       3.586221                               0.0   
1                       0.

# Data Cleaning

In [3]:
from pathlib import Path

all_csv_files = list(Path("data").rglob("*.csv"))
print("Found:", len(all_csv_files), "files")
for f in all_csv_files[:10]:
    print(f)


Found: 1504 files
data\USA_AK_Anchorage.Intl.AP.702730_TMY3\RefBldgFullServiceRestaurantNew2004_v1.3_7.1_8A_USA_AK_FAIRBANKS.csv
data\USA_AK_Anchorage.Intl.AP.702730_TMY3\RefBldgHospitalNew2004_v1.3_7.1_8A_USA_AK_FAIRBANKS.csv
data\USA_AK_Anchorage.Intl.AP.702730_TMY3\RefBldgLargeHotelNew2004_v1.3_7.1_8A_USA_AK_FAIRBANKS.csv
data\USA_AK_Anchorage.Intl.AP.702730_TMY3\RefBldgLargeOfficeNew2004_v1.3_7.1_8A_USA_AK_FAIRBANKS.csv
data\USA_AK_Anchorage.Intl.AP.702730_TMY3\RefBldgMediumOfficeNew2004_v1.3_7.1_8A_USA_AK_FAIRBANKS.csv
data\USA_AK_Anchorage.Intl.AP.702730_TMY3\RefBldgMidriseApartmentNew2004_v1.3_7.1_8A_USA_AK_FAIRBANKS.csv
data\USA_AK_Anchorage.Intl.AP.702730_TMY3\RefBldgOutPatientNew2004_v1.3_7.1_8A_USA_AK_FAIRBANKS.csv
data\USA_AK_Anchorage.Intl.AP.702730_TMY3\RefBldgPrimarySchoolNew2004_v1.3_7.1_8A_USA_AK_FAIRBANKS.csv
data\USA_AK_Anchorage.Intl.AP.702730_TMY3\RefBldgQuickServiceRestaurantNew2004_v1.3_7.1_8A_USA_AK_FAIRBANKS.csv
data\USA_AK_Anchorage.Intl.AP.702730_TMY3\RefBldg

In [4]:
import pandas as pd
from pathlib import Path

all_csv_files = list(Path("data").rglob("*.csv"))
print("Found:", len(all_csv_files), "files")

dfs = []
for f in all_csv_files:
    try:
        df = pd.read_csv(f)

        if df.empty:
            print("⚠️ Skipping empty file:", f)
            continue

        # Ensure Date/Time exists
        if "Date/Time" in df.columns:
            df["Date/Time"] = pd.to_datetime(df["Date/Time"], errors="coerce")

        # Safe energy column creation
        df["energy_consumption"] = (
            df.get("Electricity:Facility [kW](Hourly)", pd.Series([0]*len(df))) +
            df.get("Gas:Facility [kW](Hourly)", pd.Series([0]*len(df)))
        )

        # Add metadata
        df["station_folder"] = f.parent.name
        df["file_name"] = f.name

        dfs.append(df)

    except Exception as e:
        print("❌ Error reading", f, ":", e)

# Concatenate while aligning columns
if dfs:
    data = pd.concat(dfs, ignore_index=True, sort=False)
    print("✅ Final shape:", data.shape)
    print("✅ Columns:", list(data.columns))
else:
    print("❌ No valid DataFrames created!")


Found: 1504 files


C:\Users\gupta\AppData\Local\Temp\ipykernel_5096\200440568.py:18: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["Date/Time"] = pd.to_datetime(df["Date/Time"], errors="coerce")
C:\Users\gupta\AppData\Local\Temp\ipykernel_5096\200440568.py:18: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["Date/Time"] = pd.to_datetime(df["Date/Time"], errors="coerce")
C:\Users\gupta\AppData\Local\Temp\ipykernel_5096\200440568.py:18: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["Date/Time"] = pd.to_datetime(df["Date/Time"], errors="coerce")
C:\Users\gupta\AppData\Local\Temp\ipykernel

✅ Final shape: (13175040, 16)
✅ Columns: ['Date/Time', 'Electricity:Facility [kW](Hourly)', 'Fans:Electricity [kW](Hourly)', 'Cooling:Electricity [kW](Hourly)', 'Heating:Electricity [kW](Hourly)', 'InteriorLights:Electricity [kW](Hourly)', 'InteriorEquipment:Electricity [kW](Hourly)', 'Gas:Facility [kW](Hourly)', 'Heating:Gas [kW](Hourly)', 'InteriorEquipment:Gas [kW](Hourly)', 'Water Heater:WaterSystems:Gas [kW](Hourly)', 'energy_consumption', 'station_folder', 'file_name', 'Electricity:Facility [kW](Monthly)', 'Gas:Facility [kW](Monthly)']


# Feature Engineering

In [5]:
# Convert timestamp to datetime
data["timestamp"] = pd.to_datetime(data["timestamp"])

# Extract time-based features
data["hour"] = data["timestamp"].dt.hour
data["day"] = data["timestamp"].dt.day
data["weekday"] = data["timestamp"].dt.weekday
data["month"] = data["timestamp"].dt.month

KeyError: 'timestamp'

# Correlation Analysis

In [ ]:
import matplotlib.pyplot as plt

corr = data[["energy_consumption", "temperature", "humidity"]].corr()
plt.imshow(corr, cmap="coolwarm", interpolation="nearest")
plt.xticks(range(len(corr)), corr.columns, rotation=45)
plt.yticks(range(len(corr)), corr.columns)
plt.colorbar()
plt.show()

# Visualization

Plot daily/weekly energy trends.
Compare different building types (office, hospital, school).
Compare different weather stations.

# Forecasting

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

X = data[["temperature", "humidity", "hour", "weekday", "month"]]
y = data["energy_consumption"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

model = LinearRegression()
model.fit(X_train, y_train)

print("R² Score:", model.score(X_test, y_test))